In [25]:
import numpy as np
from nanowire.optics.simulate import Simulator
from nanowire.optics.utils.utils import setup_sim
from nanowire.optics.utils.config import Config
import scipy.constants as consts
import scipy.integrate as intg

# Introduction

This notebook is intended to check the consistency between 3 different methods for calculating the absorption of a layer in S4.
The methods are as follows

1. Using the GetPowerFlux method and taking the difference between what enters the top and leaves the bottom. 
2. Using the GetVolumeIntegral function
3. Integrate the raw field output numerically

All of these methods should agree with one another, so let's verify this. Below I just set up the sim so it's ready for all subsequent computations

In [2]:
conf = Config('GaAs_slab.yml')
sim = Simulator(conf)
sim = setup_sim(sim)

In [3]:
fluxes = sim.get_fluxes()

# Poynting Flux Method

In [5]:
summed_absorbed_power = 0
abs_dict_fluxmethod = {}
for layer, (forw, back) in fluxes.items():
    if '_bottom' in layer:
        continue
    incident_power = np.absolute(forw)
    reflected_power = np.absolute(back)
    print('-'*25)
    print('Layer: {}'.format(layer))
    print('Incident Power: {}'.format(incident_power))
    print('Reflected Power: {}'.format(reflected_power))
    bottom = layer+'_bottom'
    transmitted_power = np.absolute(fluxes[bottom][0])
    bottom_reflected_power = np.absolute(fluxes[bottom][1])
    print('Transmitted Power: {}'.format(transmitted_power))
    print('Backward_bottom: {}'.format(bottom_reflected_power))
    #absorbed = incident_power + bottom_reflected_power - transmitted_power - reflected_power
    flux_top = np.absolute(forw+back)
    flux_bottom = np.absolute(fluxes[bottom][0]+fluxes[bottom][1]) 
    absorbed = flux_top - flux_bottom
    abs_dict_fluxmethod[layer] = absorbed
    print('Absorbed in Layer: {}'.format(absorbed))
    summed_absorbed_power += absorbed
#print('Summed Absorption= {}'.format(np.absolute(summed_absorbed_power)))
print('-'*25)
print('Summed Absorption= {}'.format(summed_absorbed_power))
print('GaAs Absorption = {}'.format(abs_dict_fluxmethod['Substrate']))

-------------------------
Layer: Substrate
Incident Power: 1.7579423702612038
Reflected Power: 0.09946230970688712
Transmitted Power: 0.712492733717254
Backward_bottom: 0.2362704159417706
Absorbed in Layer: 1.1826821460870796
-------------------------
Layer: Below_Air
Incident Power: 0.4764238383744144
Reflected Power: 0.0
Transmitted Power: 0.4764238383744144
Backward_bottom: 0.0
Absorbed in Layer: 0.0
-------------------------
Layer: Air
Incident Power: 2.0
Reflected Power: 0.34089401553844645
Transmitted Power: 2.0
Backward_bottom: 0.34089401553844645
Absorbed in Layer: 0.0
-------------------------
Summed Absorption= 1.1826821460870796
GaAs Absorption = 1.1826821460870796


# GetVolumeIntegral Method

We use the provided S4 function to compute

\begin{equation}
    \int \epsilon | \vec{E} | ^2 dV
\end{equation}
  
where the integral is over the volume of the layer. The total absorbed power is 

\begin{equation}
    P_{abs} = \frac{\omega}{2} \int \epsilon | \vec{E} | ^2 dV
\end{equation}

Where here we need to convert $\omega$ into the Lorentz-Heaviside units used by S4.

In [14]:
int_result = sim.s4.GetLayerVolumeIntegral(Layer="Substrate", Quantity="E")
epsilon = sim._get_epsilon('/home/kyle_robertson/software/nanowire/nanowire/NK/006_GaAs_nk_Walker_modified_Hz.txt')
print(epsilon)
si_freq = sim.conf[('Simulation', 'params', 'frequency', 'value')]
base_unit = sim.conf[('Simulation', 'base_unit')]
c_conv = consts.c / base_unit 
f_conv = si_freq / c_conv
print("Pabs = {}".format(.5*f_conv*int_result))

(13.767280000000001+0.8017920000000001j)
Pabs = (1.618750336635752-1.1898315158680083e-17j)


# Integrate Raw Fields

This approach collects the raw fields from S4 and integrates them over the volume of the layer

In [15]:
sim.get_field()

DEBUG:nanowire.optics.simulate:Computing fields ...
DEBUG:nanowire.optics.simulate:Computing up to depth of 0.5 microns
DEBUG:nanowire.optics.simulate:Finished computing fields!


In [30]:
Ex = np.absolute(sim.data['Ex'])
Ey = np.absolute(sim.data['Ey'])
Ez = np.absolute(sim.data['Ez'])
normE = np.sqrt(Ex**2 + Ey**2 + Ex**2)
zsamps = sim.conf[('Simulation', 'z_samples')]
xsamps = sim.conf[('Simulation', 'x_samples')]
ysamps = sim.conf[('Simulation', 'y_samples')]
height = sim.get_height()
z_vals = np.linspace(0, height, zsamps)
x_vals = np.linspace(0, sim.period, xsamps)
y_vals = np.linspace(0, sim.period, ysamps)
z_integral = intg.trapz(normE, x=z_vals, axis=0)
x_integral = intg.trapz(z_integral, x=x_vals, axis=0)
y_integral = intg.trapz(x_integral, x=y_vals, axis=0)
epsilon = sim._get_epsilon('/home/kyle_robertson/software/nanowire/nanowire/NK/006_GaAs_nk_Walker_modified_Hz.txt')
print(epsilon)
si_freq = sim.conf[('Simulation', 'params', 'frequency', 'value')]
base_unit = sim.conf[('Simulation', 'base_unit')]
c_conv = consts.c / base_unit 
f_conv = si_freq / c_conv
print("Pabs = {}".format(.5*f_conv*epsilon.imag*y_integral))

(13.767280000000001+0.8017920000000001j)
Pabs = 0.011723507855551719
